# 1. CARGO DATAFRAMES BASE

In [1]:
import pandas as pd
import os
import numpy as np
from datetime import datetime, timedelta

ruta_export = r"C:\Users\MSII7\Documents\Jasef\Desercion\export"
ruta_output = r"C:\Users\MSII7\Documents\Jasef\Desercion\output"
ruta_sources = r"C:\Users\MSII7\Documents\Jasef\Desercion\sources"

df_matriculados_2024_1 = pd.read_csv(os.path.join(ruta_export,'Matriculados_2024_1.csv'))
df_matriculados_2023_2 = pd.read_csv(os.path.join(ruta_export,'Matriculados_2023_2.csv'))
df_matriculados_2023_1 = pd.read_csv(os.path.join(ruta_export,'Matriculados_2023_1.csv'))
df_matriculados_2022_2 = pd.read_csv(os.path.join(ruta_export,'Matriculados_2022_2.csv'))
df_matriculados_2022_1 = pd.read_csv(os.path.join(ruta_export,'Matriculados_2022_1.csv'))


df_desertores_2023_2 = pd.read_csv(os.path.join(ruta_export,'Desertores_2023_2.csv'))

df_variables = pd.read_csv(os.path.join(ruta_export,'DataMaestra_Estudiante.csv'))

df_notas_2023_2 = pd.read_csv(os.path.join(ruta_sources,'DATA_DETALLE_NOTAS-2023-2.csv'))

df_colegio_procedencia = pd.read_csv(
    os.path.join(ruta_sources, 'DATA_COLEGIO_PROCEDENCIA.csv'),
    header=None,  # Indica que el archivo no tiene cabeceras
    names=['IdAlumno', 'Colegio', 'TipoColegio']  # Especificar las cabeceras manualmente
)
df_colegio_procedencia = df_colegio_procedencia.drop_duplicates(subset='IdAlumno')

df_pagos_2023 = pd.read_csv(os.path.join(ruta_sources,'pagos cachimbos 2023-2.csv'))


nota_aprobatoria = 13
notas_desaprobadas = 7
periodo = '2023-2'

# CALCULO VARIABLES NOTAS

In [2]:
# Agrupar por IdAlumno y contar los registros de parciales y finales desaprobados
df_cant_desaprobado_agrupado = df_notas_2023_2.groupby('IdAlumno').agg(
    CantParcialesDesaprobados=(
        'Actividad', 
        lambda x: ((x == 'EVALUACION PARCIAL') & (df_notas_2023_2.loc[x.index, 'NotaActiv'] < nota_aprobatoria)).sum()
    ),
    CantFinalesDesaprobados=(
        'Actividad', 
        lambda x: ((x == 'EVALUACION FINAL') & (df_notas_2023_2.loc[x.index, 'NotaActiv'] < nota_aprobatoria)).sum()
    ),
    # Columna auxiliar para contar las evaluaciones diferentes de PARCIAL y FINAL desaprobadas
    CantNotasDesaprobadasNoParcNoFin=(
        'Actividad', 
        lambda x: ((~x.isin(['EVALUACION PARCIAL', 'EVALUACION FINAL', 'EVALUACION DIAGNOSTICA'])) & (df_notas_2023_2.loc[x.index, 'NotaActiv'] < nota_aprobatoria)).sum()
    )
).reset_index()

# Crear la columna 'ExcedeNotasDesaprobadasNoParcNoFin'
df_cant_desaprobado_agrupado['ExcedeNotasDesaprobadasNoParcNoFin'] = df_cant_desaprobado_agrupado['CantNotasDesaprobadasNoParcNoFin'].apply(
    lambda x: 'SI' if x >= notas_desaprobadas else 'NO'
)

# Eliminar la columna 'CantNotasDesaprobadasNoParcNoFin' 
df_cant_desaprobado_agrupado.drop(columns=['CantNotasDesaprobadasNoParcNoFin'], inplace=True)

# Calcular la nota final para cada curso por alumno
df_notas_2023_2['NotaFinalCurso'] = np.where(
    df_notas_2023_2['Actividad'] != 'EVALUACION DIAGNOSTICA',
    (df_notas_2023_2['Peso'] * df_notas_2023_2['NotaActiv']) / 100,
    np.nan  # Ignoramos las actividades diagnósticas
)

# Agrupar por IdAlumno y Curso para obtener la nota final por curso
df_notas_finales_por_curso = df_notas_2023_2.groupby(['IdAlumno', 'Curso']).agg(
    NotaFinalCurso=('NotaFinalCurso', 'sum')  # Sumar las notas ponderadas por curso
).reset_index()

# Redondeo las notas a su entero más cercano para el posterior conteo 
df_notas_finales_por_curso['NotaFinalCurso'] = df_notas_finales_por_curso['NotaFinalCurso'].round()

# Agrupar por IdAlumno y contar los cursos desaprobados
df_cursos_desaprobados = df_notas_finales_por_curso.groupby('IdAlumno').agg(
    CantCursosDesaprobados=(
        'NotaFinalCurso', 
        lambda x: (x < nota_aprobatoria).sum()
    ),
    CantCursos=(
        'NotaFinalCurso', 
        'count'  # Cuenta la cantidad de cursos
    )
).reset_index()

# Creamos la columna ExcedeMitadCursosDesaprobados
df_cursos_desaprobados['ExcedeMitadCursosDesaprobados'] = df_cursos_desaprobados.apply(
    lambda row: 'SI' if (row['CantCursosDesaprobados'] / row['CantCursos']) >= 0.5 else 'NO', axis=1
)

# Eliminar la columna 'CantNotasDesaprobadasNoParcNoFin' 
df_cursos_desaprobados.drop(columns=['CantCursosDesaprobados','CantCursos'], inplace=True)

# Unir este resultado con el DataFrame anterior
df_cant_desaprobado_agrupado = df_cant_desaprobado_agrupado.merge(
    df_cursos_desaprobados, 
    on='IdAlumno', 
    how='left'
)

df_cant_desaprobado_agrupado = df_cant_desaprobado_agrupado.merge(
    df_notas_2023_2[['IdAlumno', 'ProgramaAlu']].drop_duplicates(),  # Tomar IdAlumno y ProgramaAlu sin duplicados
    on='IdAlumno',
    how='left'
)

# Visualizar el DataFrame final
print(df_cant_desaprobado_agrupado.head())

    IdAlumno  CantParcialesDesaprobados  CantFinalesDesaprobados  \
0  100002894                          0                        0   
1  100003558                          3                        3   
2  100003932                          1                        0   
3  100003946                          2                        3   
4  100003989                          0                        0   

  ExcedeNotasDesaprobadasNoParcNoFin ExcedeMitadCursosDesaprobados  \
0                                 NO                            NO   
1                                 NO                            SI   
2                                 NO                            NO   
3                                 NO                            NO   
4                                 NO                            NO   

                     ProgramaAlu  
0          Nutrición y Dietética  
1  Medic Veterinaria y Zootecnia  
2                Medicina Humana  
3  Medic Veterinaria y Zootecn

# CALCULO DE VARIABLES DE PAGO

In [3]:
df_pagos_2023['FECHA_VENCIMIENTO'] = pd.to_datetime(df_pagos_2023['FECHA_VENCIMIENTO'])
df_pagos_2023['FECHA_TRANSACCION'] = pd.to_datetime(df_pagos_2023['FECHA_TRANSACCION'])

# Paso 1: Agrupar por IdAlumno y contar la cantidad de pagos (CantArmadas)
df_agrupado = df_pagos_2023.groupby('IdAlumno').agg(
    CantArmadas=('IdAlumno', 'size')
).reset_index()

# Paso 2: Calcular CantArmadasRetraso7dias (transacciones con retraso mayor a 7 días)
df_pagos_2023['Retraso'] = (df_pagos_2023['FECHA_TRANSACCION'] - df_pagos_2023['FECHA_VENCIMIENTO']).dt.days
df_pagos_2023['RetrasoMayor7Dias'] = df_pagos_2023['Retraso'] > 7

# Agrupar por IdAlumno y contar las transacciones con retraso mayor a 7 días
df_retraso = df_pagos_2023.groupby('IdAlumno').agg(
    CantArmadasRetraso7dias=('RetrasoMayor7Dias', 'sum')
).reset_index()

# Paso 3: Unir ambos DataFrames (df_agrupado y df_retraso)
df_resultado_pagos = pd.merge(df_agrupado, df_retraso, on='IdAlumno', how='left')

# Paso 4: Crear el nuevo campo con la lógica Si/No
df_resultado_pagos['ExcedePagosAtrasados'] = df_resultado_pagos.apply(
    lambda row: 1 if row['CantArmadasRetraso7dias'] >= (row['CantArmadas'] / 2) else 0,
    axis=1
)

print(df_resultado_pagos.head())

    IdAlumno  CantArmadas  CantArmadasRetraso7dias  ExcedePagosAtrasados
0  100004070            2                        2                     1
1  100008630            5                        4                     1
2  100009785            5                        4                     1
3  100011191            5                        1                     0
4  100012563            5                        2                     0


# IDENTIFICO LOS ESTUDIANTES A PREDECIR

In [4]:
# Unir todos los periodos anteriores en un solo dataframe
df_matriculados_anteriores = pd.concat([df_matriculados_2023_1, df_matriculados_2022_2, df_matriculados_2022_1])

In [5]:
df_nuevos_matriculados_2023_2 = df_matriculados_2023_2[~df_matriculados_2023_2['IdAlumno'].isin(df_matriculados_anteriores['IdAlumno'])]

In [6]:
# Filtrar el DataFrame para encontrar el registro específico
resultado = df_nuevos_matriculados_2023_2[df_nuevos_matriculados_2023_2['IdAlumno'] == 100118394]

# Verifica el resultado
print(resultado)

Empty DataFrame
Columns: [CicloLectivoDesCorta, IdAlumno, Anio]
Index: []


In [7]:
print(df_nuevos_matriculados_2023_2.shape)

(1205, 3)


In [8]:
print(df_variables.isnull().sum())

IdAlumno                   0
Genero                     0
FechaNacimiento            0
Distrito                   0
Provincia                  0
Departamento               0
FamiliarResponsable    20211
AnioNacimiento             0
dtype: int64


In [9]:
#Variables CRM
df = pd.merge(df_nuevos_matriculados_2023_2, df_variables, on='IdAlumno', how='left')

#Variables Notas
df = pd.merge(df, df_cant_desaprobado_agrupado, on='IdAlumno', how='left')

#Colegio Procedencia
df = pd.merge(df, df_colegio_procedencia, on='IdAlumno', how='left')

#Variables Pagos
df = pd.merge(df, df_resultado_pagos, on='IdAlumno', how='left')

# columna objetivo
df['Desercion'] = df['IdAlumno'].isin(df_desertores_2023_2['IdAlumno']).astype(int)

print(df.head())

  CicloLectivoDesCorta   IdAlumno  Anio Genero FechaNacimiento Distrito  \
0               2023-2  100004070  2023    NaN             NaN      NaN   
1               2023-2  100008630  2023    NaN             NaN      NaN   
2               2023-2  100009785  2023    NaN             NaN      NaN   
3               2023-2  100011191  2023    NaN             NaN      NaN   
4               2023-2  100012563  2023    NaN             NaN      NaN   

  Provincia Departamento FamiliarResponsable  AnioNacimiento  ...  \
0       NaN          NaN                 NaN             NaN  ...   
1       NaN          NaN                 NaN             NaN  ...   
2       NaN          NaN                 NaN             NaN  ...   
3       NaN          NaN                 NaN             NaN  ...   
4       NaN          NaN                 NaN             NaN  ...   

   CantFinalesDesaprobados  ExcedeNotasDesaprobadasNoParcNoFin  \
0                        0                                  SI   
1 

In [10]:
print(df.shape)

(1205, 21)


### RELLENO LOS QUE NO TIENEN GENERO



In [11]:
# Calculamos la cuenta de cada género y de los valores nulos
conteo_generos = df['Genero'].value_counts(dropna=False)
count_f = conteo_generos.get('F', 0)  # Cantidad de F
count_m = conteo_generos.get('M', 0)  # Cantidad de M
total_blancos = conteo_generos.get(np.nan, 0)  # Cantidad de valores nulos

# Ahora, calculamos la distribución equitativa
if total_blancos > 0:
    # Proporciones de cada género
    proporciones_f = count_f / (count_f + count_m)
    proporciones_m = count_m / (count_f + count_m)

    # Cálculo de cuántos géneros se asignarán a los valores nulos
    asignacion_f = int(total_blancos * proporciones_f)
    asignacion_m = total_blancos - asignacion_f  # Lo que queda se asigna a M

    # Rellenar los valores nulos en el DataFrame
    df.loc[df['Genero'].isna(), 'Genero'] = ['F'] * asignacion_f + ['M'] * asignacion_m


print(df.isnull().sum())

CicloLectivoDesCorta                    0
IdAlumno                                0
Anio                                    0
Genero                                  0
FechaNacimiento                        90
Distrito                               90
Provincia                              90
Departamento                           90
FamiliarResponsable                   742
AnioNacimiento                         90
CantParcialesDesaprobados               0
CantFinalesDesaprobados                 0
ExcedeNotasDesaprobadasNoParcNoFin      0
ExcedeMitadCursosDesaprobados           0
ProgramaAlu                             0
Colegio                                 0
TipoColegio                             0
CantArmadas                            28
CantArmadasRetraso7dias                28
ExcedePagosAtrasados                   28
Desercion                               0
dtype: int64


### RELLENO LOS QUE NO TIENEN FECHA DE NACIMIENTO

In [12]:
# Contar cuántos valores nulos hay en FechaNacimiento
n = df['FechaNacimiento'].isnull().sum()

# Función para generar fechas aleatorias en el formato 'YYYY-MM-DD'
def generar_fecha_aleatoria(n):
    fechas = []
    for _ in range(n):
        anio = np.random.choice([2006, 2007])  # Años para tener 16 o 17 años en 2023
        mes = np.random.randint(1, 13)  # Mes de 1 a 12
        dia = np.random.randint(1, 29)  # Día de 1 a 28 (para simplificar)
        fecha = f"{anio}-{mes:02d}-{dia:02d}"  # Formato 'YYYY-MM-DD'
        fechas.append(fecha)
    return fechas

# Generar fechas aleatorias y rellenar los nulos
fechas_aleatorias = generar_fecha_aleatoria(n)
df.loc[df['FechaNacimiento'].isnull(), 'FechaNacimiento'] = fechas_aleatorias

# Obtener el año de nacimiento de los nulos en FechaNacimiento
df.loc[df['AnioNacimiento'].isnull(), 'AnioNacimiento'] = df['FechaNacimiento'].str.split('-').str[0].astype(int)

#print(df)
print(df.isnull().sum())


CicloLectivoDesCorta                    0
IdAlumno                                0
Anio                                    0
Genero                                  0
FechaNacimiento                         0
Distrito                               90
Provincia                              90
Departamento                           90
FamiliarResponsable                   742
AnioNacimiento                          0
CantParcialesDesaprobados               0
CantFinalesDesaprobados                 0
ExcedeNotasDesaprobadasNoParcNoFin      0
ExcedeMitadCursosDesaprobados           0
ProgramaAlu                             0
Colegio                                 0
TipoColegio                             0
CantArmadas                            28
CantArmadasRetraso7dias                28
ExcedePagosAtrasados                   28
Desercion                               0
dtype: int64


### RELLENO LOS QUE NO TIENEN DISTRITO

In [13]:
# Lista de distritos para rellenar
distritos = ["ATE", "VILLA EL SALVADOR", "COMAS", "CHORRILLOS", "SAN MARTÍN DE PORRES"]

# Contar cuántos valores nulos hay en el campo 'Distrito'
nulos_distrito = df['Distrito'].isnull().sum()

# Calcular cuántos registros de cada distrito se necesitan
if nulos_distrito > 0:
    # Calcular cuántos distritos asignar a cada uno
    asignaciones = [nulos_distrito // len(distritos)] * len(distritos)
    
    # Distribuir los restantes de manera aleatoria entre los distritos
    for i in range(nulos_distrito % len(distritos)):
        asignaciones[i] += 1

    # Crear una lista con los distritos asignados
    distritos_asignados = []
    for distrito, cantidad in zip(distritos, asignaciones):
        distritos_asignados.extend([distrito] * cantidad)

    # Mezclar aleatoriamente la lista para distribuir uniformemente
    np.random.shuffle(distritos_asignados)

    # Rellenar los valores nulos en el DataFrame
    df.loc[df['Distrito'].isnull(), 'Distrito'] = distritos_asignados

# Mostrar el DataFrame resultante
print(df.isnull().sum())

CicloLectivoDesCorta                    0
IdAlumno                                0
Anio                                    0
Genero                                  0
FechaNacimiento                         0
Distrito                                0
Provincia                              90
Departamento                           90
FamiliarResponsable                   742
AnioNacimiento                          0
CantParcialesDesaprobados               0
CantFinalesDesaprobados                 0
ExcedeNotasDesaprobadasNoParcNoFin      0
ExcedeMitadCursosDesaprobados           0
ProgramaAlu                             0
Colegio                                 0
TipoColegio                             0
CantArmadas                            28
CantArmadasRetraso7dias                28
ExcedePagosAtrasados                   28
Desercion                               0
dtype: int64


### RELLENO LOS QUE NO TIENEN PROVINCIA NI DEPARTAMENTO

In [14]:
df['Provincia'] = df['Provincia'].fillna('LIMA')
df['Departamento'] = df['Departamento'].fillna('LIMA')

print(df.isnull().sum())

CicloLectivoDesCorta                    0
IdAlumno                                0
Anio                                    0
Genero                                  0
FechaNacimiento                         0
Distrito                                0
Provincia                               0
Departamento                            0
FamiliarResponsable                   742
AnioNacimiento                          0
CantParcialesDesaprobados               0
CantFinalesDesaprobados                 0
ExcedeNotasDesaprobadasNoParcNoFin      0
ExcedeMitadCursosDesaprobados           0
ProgramaAlu                             0
Colegio                                 0
TipoColegio                             0
CantArmadas                            28
CantArmadasRetraso7dias                28
ExcedePagosAtrasados                   28
Desercion                               0
dtype: int64


### CREO CAMPO EDAD

In [15]:
df['Edad'] = df['Anio'] - df['AnioNacimiento']
print(df.isnull().sum())

CicloLectivoDesCorta                    0
IdAlumno                                0
Anio                                    0
Genero                                  0
FechaNacimiento                         0
Distrito                                0
Provincia                               0
Departamento                            0
FamiliarResponsable                   742
AnioNacimiento                          0
CantParcialesDesaprobados               0
CantFinalesDesaprobados                 0
ExcedeNotasDesaprobadasNoParcNoFin      0
ExcedeMitadCursosDesaprobados           0
ProgramaAlu                             0
Colegio                                 0
TipoColegio                             0
CantArmadas                            28
CantArmadasRetraso7dias                28
ExcedePagosAtrasados                   28
Desercion                               0
Edad                                    0
dtype: int64


In [16]:
# Definir las condiciones y sus correspondientes etiquetas
condiciones = [
    (df['Edad'] <= 18),
    (df['Edad'] > 18) & (df['Edad'] < 25),
    (df['Edad'] >= 25)
]

etiquetas = ['MenosDe18', 'Entre18y25', 'Mayor25']

# Crear la nueva columna 'GrupoEdad' con las etiquetas según las condiciones, usando un valor por defecto como ''
df['GrupoEdad'] = np.select(condiciones, etiquetas, default='')

# Verificar los resultados
print(df[['Edad', 'GrupoEdad']].head())

   Edad  GrupoEdad
0  17.0  MenosDe18
1  16.0  MenosDe18
2  16.0  MenosDe18
3  17.0  MenosDe18
4  16.0  MenosDe18


### RELLENO EL FAMILIAR RESPONSABLE

In [17]:
# Filas donde FamiliarResponsable es nulo
mask = df['FamiliarResponsable'].isnull()

# Aplicar la condición solo a esas filas
df.loc[mask, 'FamiliarResponsable'] = df.loc[mask, 'Edad'].apply(lambda x: 'Apoderado' if x <= 18 else 'Alumno')

print(df.isnull().sum())

CicloLectivoDesCorta                   0
IdAlumno                               0
Anio                                   0
Genero                                 0
FechaNacimiento                        0
Distrito                               0
Provincia                              0
Departamento                           0
FamiliarResponsable                    0
AnioNacimiento                         0
CantParcialesDesaprobados              0
CantFinalesDesaprobados                0
ExcedeNotasDesaprobadasNoParcNoFin     0
ExcedeMitadCursosDesaprobados          0
ProgramaAlu                            0
Colegio                                0
TipoColegio                            0
CantArmadas                           28
CantArmadasRetraso7dias               28
ExcedePagosAtrasados                  28
Desercion                              0
Edad                                   0
GrupoEdad                              0
dtype: int64


### Relleno el tipo de colegio de procedencia

In [18]:
df['TipoColegio'] = df_colegio_procedencia['TipoColegio'].replace({'RELG': 'PRIV', 'OTHR': 'PUBL'}).fillna('PUBL')

print(df.isnull().sum())

CicloLectivoDesCorta                   0
IdAlumno                               0
Anio                                   0
Genero                                 0
FechaNacimiento                        0
Distrito                               0
Provincia                              0
Departamento                           0
FamiliarResponsable                    0
AnioNacimiento                         0
CantParcialesDesaprobados              0
CantFinalesDesaprobados                0
ExcedeNotasDesaprobadasNoParcNoFin     0
ExcedeMitadCursosDesaprobados          0
ProgramaAlu                            0
Colegio                                0
TipoColegio                            1
CantArmadas                           28
CantArmadasRetraso7dias               28
ExcedePagosAtrasados                  28
Desercion                              0
Edad                                   0
GrupoEdad                              0
dtype: int64


In [19]:
print(df.shape)
df.to_csv('cachimbos.csv')

(1205, 23)


In [20]:
variables_modelo = ['IdAlumno',
                    'Edad', 
                    'Genero', 
                    'Distrito', 
                    'Provincia', 
                    'Departamento', 
                    'FamiliarResponsable', 
                    'CantParcialesDesaprobados', 
                    'CantFinalesDesaprobados',
                    'ExcedeNotasDesaprobadasNoParcNoFin',
                    'ExcedeMitadCursosDesaprobados',
                    'GrupoEdad',
                    'ExcedePagosAtrasados',
                    'TipoColegio', 
                    'Desercion']
df_modelo = df[variables_modelo]

# Separar IdAlumno
id_alumno = df_modelo['IdAlumno']

# Convertir variables categóricas en variables dummy
df_modelo = pd.get_dummies(df_modelo.drop(columns=['IdAlumno']), 
                            columns=['Genero', 
                                     'Distrito', 
                                     'Provincia', 
                                     'Departamento', 
                                     'FamiliarResponsable', 
                                     'CantParcialesDesaprobados', 
                                     'CantFinalesDesaprobados',
                                     'ExcedeNotasDesaprobadasNoParcNoFin', 
                                     'ExcedeMitadCursosDesaprobados',
                                     'GrupoEdad', 
                                     'ExcedePagosAtrasados', 
                                     'TipoColegio'], drop_first=True)


# CREACION DEL MODELO

In [21]:
#División de los datos en entrenamiento y prueba
from sklearn.model_selection import train_test_split

X = df_modelo.drop(columns=['Desercion'])
y = df_modelo['Desercion']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [22]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix,f1_score, precision_score, recall_score
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
import shap

# Diccionario que guarda las métricas
metricas_dict = {
    "Modelo": [],
    "Tipo_Data": [],
    "Exactitud": [],
    "AUC": [],
    "F1-Score": [],
    "Precisión": [],
    "Recall": [],
    "Puntuación": []
}

predicciones_dict = {}
modelos_dict = {}  # Diccionario para guardar los modelos entrenados
shap_dict = {}

# Función para calcular métricas
def calcular_metricas(model,flg_grid_search, X_train, y_train, X_test, y_test, nombre_modelo):

    if flg_grid_search == 0: #Si no es grid search yo lo entreno
        # Entrenar el modelo con el conjunto de entrenamiento
        model.fit(X_train, y_train)

    # Hacer predicciones en el conjunto de prueba
    y_pred = model.predict(X_test)

    # Hacer predicciones en todo el conjunto de datos
    y_pred_completo = model.predict(X)

    accuracy = accuracy_score(y_test, y_pred)
    auc = roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])
    f1 = f1_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, zero_division=0)
    recall = recall_score(y_test, y_pred)


    puntuacion = (0.3 * accuracy) + (0.1 * precision) + (0.3 * recall) + (0.2 * f1) + (0.4 * auc)
    
    metricas_dict["Modelo"].append(nombre_modelo)
    metricas_dict["Tipo_Data"].append("Data Original")
    metricas_dict["Exactitud"].append(accuracy)
    metricas_dict["AUC"].append(auc)
    metricas_dict["F1-Score"].append(f1)
    metricas_dict["Precisión"].append(precision)
    metricas_dict["Recall"].append(recall)
    metricas_dict["Puntuación"].append((puntuacion))

    # Almacenar las predicciones en el diccionario
    predicciones_dict[nombre_modelo] = y_pred_completo

     # Guardar el modelo entrenado en el diccionario
    modelos_dict[nombre_modelo] = model  # Aquí guardamos el modelo
    


c:\Users\MSII7\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## MODELOS SIN BALANCEO

In [23]:
modelos = [
    ("Regresión Logística", LogisticRegression(max_iter=1000)),
    ("Random Forest", RandomForestClassifier(random_state=42)),
    ("XGBoost", xgb.XGBClassifier(eval_metric='logloss', random_state=42)),
    ("SVM", SVC(probability=True, random_state=42)),
    ("KNN",KNeighborsClassifier(n_neighbors=5))
]

In [24]:
# Entrenar y evaluar los modelos
for nombre_modelo, modelo in modelos:
    calcular_metricas(modelo,0,X_train,y_train, X_test, y_test, nombre_modelo)

## MODELOS CON BALANCEO

In [25]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

# Reentrenar modelos con datos balanceados
for nombre_modelo, modelo in modelos:
    calcular_metricas(modelo,0,X_resampled, y_resampled, X_test, y_test, f"{nombre_modelo} (SMOTE)")

## MODELOS CON BALANCEO Y AJUSTE DE HIPERPARAMETROS

In [26]:
from sklearn.model_selection import GridSearchCV

# Definir los hiperparámetros para cada modelo
param_grid_lr = {
    'C': [0.01, 0.1, 1, 10],
    'solver': ['lbfgs', 'liblinear']
}

param_grid_rf = {
    'n_estimators': [100, 200, 300],
    'max_depth': [5, 10, 20],
    'min_samples_split': [2, 5, 10]
}

param_grid_svm = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf'],
    'gamma': ['scale', 'auto']
}

param_grid_knn = {
    'n_neighbors': [3, 5, 7],
    'weights': ['uniform', 'distance']
}

param_grid_xgb = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7]
}

In [27]:
# Diccionario con modelos y sus respectivos grids
modelos_param_grids = [
    ("Regresión Logística", LogisticRegression(max_iter=1000), param_grid_lr),
    ("Random Forest", RandomForestClassifier(random_state=42), param_grid_rf),
    ("XGBoost", xgb.XGBClassifier(eval_metric='logloss', random_state=42), param_grid_xgb),
    ("SVM", SVC(probability=True, random_state=42), param_grid_svm),
    ("KNN", KNeighborsClassifier(), param_grid_knn)
]

In [28]:
# Realizamos la búsqueda de hiperparámetros
for nombre_modelo, modelo, param_grid in modelos_param_grids:
    grid_search = GridSearchCV(estimator=modelo, param_grid=param_grid, scoring='roc_auc', cv=5, n_jobs=-1)
    grid_search.fit(X_resampled, y_resampled)

    
    # Entrenamos con los mejores hiperparámetros
    best_model = grid_search.best_estimator_
    print(f"Mejores parámetros para {nombre_modelo}: {grid_search.best_params_}")
    
    # Calculamos métricas con los mejores hiperparámetros
    calcular_metricas(best_model,1,X_resampled,y_resampled, X_test, y_test, f"{nombre_modelo} (Optimizado y con SMOTE)")

Mejores parámetros para Regresión Logística: {'C': 10, 'solver': 'lbfgs'}
Mejores parámetros para Random Forest: {'max_depth': 20, 'min_samples_split': 2, 'n_estimators': 300}
Mejores parámetros para XGBoost: {'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 200}
Mejores parámetros para SVM: {'C': 1, 'gamma': 'scale', 'kernel': 'linear'}
Mejores parámetros para KNN: {'n_neighbors': 5, 'weights': 'distance'}


# EXPORTADO DE METRICAS

In [29]:
df_metricas = pd.DataFrame(metricas_dict)

df_metricas = df_metricas.sort_values(by="Puntuación", ascending=False)
df_metricas.to_csv(os.path.join(ruta_output,'Modelo_Desercion_Metricas.csv'))
print(df_metricas)

                                          Modelo      Tipo_Data  Exactitud  \
13                  SVM (Optimizado y con SMOTE)  Data Original   0.812155   
5                    Regresión Logística (SMOTE)  Data Original   0.784530   
10  Regresión Logística (Optimizado y con SMOTE)  Data Original   0.792818   
8                                    SVM (SMOTE)  Data Original   0.759669   
11        Random Forest (Optimizado y con SMOTE)  Data Original   0.762431   
2                                        XGBoost  Data Original   0.762431   
0                            Regresión Logística  Data Original   0.765193   
12              XGBoost (Optimizado y con SMOTE)  Data Original   0.740331   
6                          Random Forest (SMOTE)  Data Original   0.751381   
1                                  Random Forest  Data Original   0.762431   
7                                XGBoost (SMOTE)  Data Original   0.729282   
9                                    KNN (SMOTE)  Data Original 

In [30]:
# Seleccionar el modelo ganador basado en la puntuación más alta
modelo_ganador_idx = df_metricas['Puntuación'].idxmax()
modelo_ganador = df_metricas.loc[modelo_ganador_idx]
nombre_modelo_ganador = modelo_ganador["Modelo"]

print(f"El modelo ganador es: {nombre_modelo_ganador}")
print(modelo_ganador)

El modelo ganador es: SVM (Optimizado y con SMOTE)
Modelo        SVM (Optimizado y con SMOTE)
Tipo_Data                    Data Original
Exactitud                         0.812155
AUC                               0.816501
F1-Score                          0.693694
Precisión                         0.762376
Recall                            0.636364
Puntuación                        0.976132
Name: 13, dtype: object


In [31]:
print(len(predicciones_dict[nombre_modelo_ganador]))

1205


In [96]:
# Crear DataFrame con IdAlumno y las predicciones
predicciones_df = pd.DataFrame({
    'IdAlumno': id_alumno.loc[X.index],  # Usar índices de X
    'Prediccion': predicciones_dict[nombre_modelo_ganador]
})

dataset_base = df[variables_modelo]

resultado_final = pd.merge(predicciones_df,dataset_base,on='IdAlumno',how='left') 

print(resultado_final.shape)
print(resultado_final)

resultado_final.to_csv(os.path.join(ruta_output,'Prediccion.csv'), index=False)

(1205, 16)
        IdAlumno  Prediccion  Edad Genero           Distrito Provincia  \
0      100004070           0  17.0      F  VILLA EL SALVADOR      LIMA   
1      100008630           0  16.0      F                ATE      LIMA   
2      100009785           1  16.0      F         CHORRILLOS      LIMA   
3      100011191           1  17.0      F              COMAS      LIMA   
4      100012563           0  16.0      F              COMAS      LIMA   
...          ...         ...   ...    ...                ...       ...   
1200   180000324           1  17.0      M         CHORRILLOS      LIMA   
1201   180000558           1  17.0      M  VILLA EL SALVADOR      LIMA   
1202   180000790           0  17.0      M  VILLA EL SALVADOR      LIMA   
1203  4200710473           0  17.0      M         CHORRILLOS      LIMA   
1204  4200810288           1  17.0      M              COMAS      LIMA   

     Departamento FamiliarResponsable  CantParcialesDesaprobados  \
0            LIMA           Apod

In [35]:
obj_modelo_ganador = modelos_dict[nombre_modelo_ganador]
print(obj_modelo_ganador)

SVC(C=1, kernel='linear', probability=True, random_state=42)


In [57]:
# Obtener coeficientes
coeficientes = obj_modelo_ganador.coef_[0]  # Para SVM lineales
importancia_features = pd.Series(coeficientes, index=X_train.columns)
importancia_features.sort_values(ascending=False, inplace=True)


# Mostrar las características más importantes
print(type(importancia_features))

<class 'pandas.core.series.Series'>


In [79]:
# Crear el explainer para el modelo SVM
explainer = shap.LinearExplainer(obj_modelo_ganador, X)

# Calcular los valores SHAP
shap_values = explainer.shap_values(X)

print(len(shap_values))


1205


In [81]:
# Convertir los valores SHAP en un DataFrame
shap_df = pd.DataFrame(shap_values, columns=X.columns)

In [85]:
# Agregar la columna 'IdAlumno'
shap_df['IdAlumno'] = id_alumno.values

In [97]:
print(shap_df)
shap_df.to_csv(os.path.join(ruta_output,'Impacto_Variables.csv'),index=False)

          Edad  Genero_M Distrito_ANDAHUAYLAS Distrito_ASIA Distrito_ATE  \
0    -0.191657 -0.055568                  0.0           0.0    -0.108473   
1    -0.274625 -0.055568                  0.0           0.0     1.247441   
2    -0.274625 -0.055568                  0.0           0.0    -0.108473   
3    -0.191657 -0.055568                  0.0           0.0    -0.108473   
4    -0.274625 -0.055568                  0.0           0.0    -0.108473   
...        ...       ...                  ...           ...          ...   
1200 -0.191657  0.103198                  0.0           0.0    -0.108473   
1201 -0.191657  0.103198                  0.0           0.0    -0.108473   
1202 -0.191657  0.103198                  0.0           0.0    -0.108473   
1203 -0.191657  0.103198                  0.0           0.0    -0.108473   
1204 -0.191657  0.103198                  0.0           0.0    -0.108473   

     Distrito_AUCALLAMA Distrito_AYACUCHO Distrito_BAGUA GRANDE  \
0              0.001

In [94]:
# Filtrar el DataFrame para el IdAlumno específico
id_alumno_especifico = 100014115
resultado_filtrado = shap_df.loc[shap_df['IdAlumno'] == id_alumno_especifico]

# Mostrar el resultado filtrado
#print(resultado_filtrado)
print(display(resultado_filtrado))


,Edad,Genero_M,Distrito_ANDAHUAYLAS,Distrito_ASIA,Distrito_ATE,Distrito_AUCALLAMA,Distrito_AYACUCHO,Distrito_BAGUA GRANDE,Distrito_BARRANCA,Distrito_BARRANCO,...,CantFinalesDesaprobados_20,CantFinalesDesaprobados_21,CantFinalesDesaprobados_28,ExcedeNotasDesaprobadasNoParcNoFin_SI,ExcedeMitadCursosDesaprobados_SI,GrupoEdad_Mayor25,GrupoEdad_MenosDe18,ExcedePagosAtrasados_1.0,TipoColegio_PUBL,IdAlumno
6,-0.274625,-0.055568,0.0,0.0,-0.108473,0.001121,0.000654,0.0,0.0,0.0,...,0.0,-0.038314,0.0,-0.000026,-0.163151,0.021839,0.045395,-0.048431,-0.044011,100014115


None
